## Faiss - Facebook AI similarity search

In [ ]:
import requests
import os

data_url = "https://raw.githubusercontent.com/jamescalam/data/main/sentence_embeddings_15K/"

# create data directory to store data
if not os.path.exists('./data'):
    os.mkdir('./data')

# download the numpy binary files (dense vectors)
for i in range(57):
    if i < 10:
        i = '0' + str(i)
    res = requests.get(data_url+f"embeddings_{i}.npy")
    with open(f'./data/embeddings_{i}.npy', 'wb') as fp:
        for chunk in res:
            fp.write(chunk)
    print('.', end='')

# and download the respective text file
res = requests.get(f"{data_url}sentences.txt")
with open(f"./data/sentences.txt", 'wb') as fp:
    for chunk in res:
        fp.write(chunk)

.........................................................

In [ ]:
import numpy as np

In [ ]:
## Note: 0 to 9 .npy file is currepted so i started from 10th to last file

for i in range(10, 57):
  if i == 10:
    with open(f'/content/data/embeddings_{i}.npy') as fp:
      sentence_embeddings = np.load(f'/content/data/embeddings_{i}.npy', allow_pickle=True)
  else:
    with open(f'/content/data/embeddings_{i}.npy') as fp:
      sentence_embeddings = np.append(
          sentence_embeddings,
          np.load(f'/content/data/embeddings_{i}.npy', allow_pickle=True), axis=0
      )


In [ ]:
sentence_embeddings.shape

(11944, 768)

In [ ]:
with open('data/sentences.txt', 'r') as f:
  lines = f.read().split('\n')

In [ ]:
lines[:5]

['A group of four children dancing in a backyard.',
 'The Conference Board said its measure of business confidence, which had fallen to 53 in the first quarter of 2003, improved to 60 in the most recent second quarter.',
 'a person eating a meal, often in a restaurant',
 'When you crossed the line, you violated the constitutional right," said Charles Weisselberg, who teaches law at the University of California, Berkeley.',
 "Ross Garber, Rowland's legal counsel, said the governor would have no comment on the condo deal."]

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.5 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 `IndexFlatL2` measures the L2 (or Euclidean) distance between all given points between our query vector, and the vectors loaded into the index.

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 45.8 MB/s eta 0:00:00


In [ ]:
# IndexFlatL2
import faiss

index = faiss.IndexFlatL2(sentence_embeddings.shape[1])    # Dimension: (11944, 768)

In [ ]:
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)

In [ ]:
index.ntotal

11944

In [ ]:
# quary vector(xq)
xq = model.encode(['someone sprint with a football'])

In [ ]:
k = 4

In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

[[9905 2026 7692 8002]]
CPU times: user 8.97 ms, sys: 0 ns, total: 8.97 ms
Wall time: 9.44 ms


In [ ]:
[f'{i}: {lines[i]}' for i in I[0]]

['9905: The large beige dog is running through the grass.',
 '2026: Little black and white calf with a tag on its ear.',
 '7692: A person is doing a trick on a skateboard',
 '8002: Two smiling young women giving the thumbs up sign.']